## Implementación de una IA para predecir intensidad de ejercicios matemáticos.

### Descripción

Para diseñar la IA se implementarán 2 métodos de de ML. Se utilizará una Red Neuronal Artificial y un Supporting Vector Machine para ver cuál de los 2 da mejor precisión al momento de obtener resultados. Lo que se busca es que cuando un nuevo jugador entre al juego, se pueda estimadar cuál sería la intensidad con la que se presentarán los ejercicios de cada dificultad. Esta intensidad se irá re-ajustando con base en el desempeño que los usuarios tengan al recibir un grupo de ejercicios.

### Librerías

* __NumPy__: Para operaciones algebráicas y arreglos multidimensionales
* __Pandas__: Para estructuras de datos fáciles de usar, con alto desempeño y varias utilidades de analítica de datos
* __Seaborn__: Para visualizaciones de datos estadísticos con interfaz profesional.
* __Scikit-Learn__: Para realizar preprocesamiento y partición de datos, utilizar SVM y validar la precisión de los modelos de ML.
* __Keras__: Para utilizar redes neuronales de una forma rápida, sin complicaciones.
* __Pickle__: Para serializar objetos. Se usará para guardar los datos que describen a la red neuronal

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
#Pre-processing, dataset partition and metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
#Deep learning library
from keras.models import Sequential
from keras.layers import Dense
#Data serialization
import pickle;

Using TensorFlow backend.


### Limpieza de datos

Antes que nada, hay que preparar los datos. Para esto vamos a cargar los datos del último backup de la base de datos, con nombre _datos_hartford_marzo.csv_.

In [2]:
#Reading the data
hartfortDF = pd.read_csv('datasets/datos_hartford_marzo.csv');
hartfortDF.head(5)

,id,edad,escuela,genero,grado,loID,problema,respuesta,respuestaJ,tiempo,tipo,userID
0,-LaQZuzKIyNwR9BjyCRb,11,Hartford International School,0,5,0,1+3=?,4,4,3.846108,1,-LaQZtfxo0QA8Ij62dk1
1,-LaQZvtdCeLWO-JIpe-5,10,Hartford International School,0,5,0,1+3=?,4,4,2.995306,1,-LaQZurPwCF55Hiv18kH
2,-LaQZw327YO_BI2Q6T4S,11,Hartford International School,0,5,0,4+2=?,6,6,4.395411,1,-LaQZtfxo0QA8Ij62dk1
3,-LaQZwUESO5MHv2WwDPU,10,Hartford International School,0,5,0,4+2=?,6,6,2.422333,1,-LaQZurPwCF55Hiv18kH
4,-LaQZxM_2u3CMfFABrsf,11,Hartford International School,0,5,0,4+3=?,7,7,5.348062,1,-LaQZtfxo0QA8Ij62dk1


Comenzaremos por obviar algunas columnas del dataset como:
* __id__: El identificador de una respuesta. El index que tiene ese registro en el dataset será más útil.
* __escuela__: Puesto que solo estamos considerando un colegio por el momento, no es necesario tenerlo.
* __tipo__: Hace referencia al tipo de pregunta. Solo se manejarán cerradas, por tanto es irrelevante.

Las variables _respuesta_ y _respuestaJ_ hacen referencia a la respuesta del ejercicio y la del jugador respectivamente. Las respuestas del jugador pueden ser iguales, diferentes o contener un _no sé_ (indicado como un - en el campo de respuestaJ). Por simplicidad para la red neuronal, se utilizará una nueva columna _isCorrect_, donde se indicará si la respuesta fue _correcta_ o _incorrecta_ con un 1 o un 0 respectivamente. Será __correcto__ si la respuesta del jugador y la del problema son las mismas, e __incorrecto__ si la respuesta del jugador es diferente o contiene un _no sé_. 

In [3]:
def isAnswerCorrect(df):
    df['respuesta'] = df['respuesta'].apply(str)
    isCorrect = (df['respuesta'] == df['respuestaJ']);
    return 1*isCorrect;

isCorrectColumn = isAnswerCorrect(hartfortDF);
hartfortDF['isCorrect'] = isCorrectColumn;
hartfortDF = hartfortDF.drop(labels=['id','escuela','problema','respuesta','respuestaJ','tipo'],axis=1)
hartfortDF.head(5)

,edad,genero,grado,loID,tiempo,userID,isCorrect
0,11,0,5,0,3.846108,-LaQZtfxo0QA8Ij62dk1,1
1,10,0,5,0,2.995306,-LaQZurPwCF55Hiv18kH,1
2,11,0,5,0,4.395411,-LaQZtfxo0QA8Ij62dk1,1
3,10,0,5,0,2.422333,-LaQZurPwCF55Hiv18kH,1
4,11,0,5,0,5.348062,-LaQZtfxo0QA8Ij62dk1,1


Estos datos presentan algunas anomalías, pues el banco de preguntas tiene 25 preguntas. El número de datos actual no corresponde a un múltiplo de 25, por tanto hay datos basura. Además, se sabe que hay un niño con edad "1" en el dataset. Esto no debería ser posible. Por tanto es necesario realizar unos ajustes al dataset completo. Solo se considerarán los niños con _edad > 4_. 

In [4]:
hartfortDF = hartfortDF.loc[lambda x: x.edad > 4]

Acto seguido, eliminará al usuario con número de ejercicios incompleto. Para saber esto se agruparán los niños por userID y se realizará un conteo de cuantos registros tiene su ID asociado. Si todas las respuestas fueron contestadas, debería aparecer un valor de 25 en todas las columnas de ese registro. La variable _uncompleteData_ tendrá la información del usuario con menos de 25 respuestas, donde tomaremos su userID y se procederá a eliminar del dataset.

In [5]:
uncompleteData = hartfortDF.groupby(['userID']).count().loc[lambda x: x.edad < 25]
hartfortDF = hartfortDF.loc[lambda x: x.userID != '-LaQlc1fC7y-HDX9BHFG']


Como eliminamos datos, es necesario reajustar los indices que tiene cada registro en el dataset.

In [6]:
hartfortDF = hartfortDF.reset_index(drop=True)

### Estableciendo medidas de rendimiento

Luego de limpiar los datos se procederá a establecer medidas de rendimiento que ayudarán al modelo a predecir los siguientes valores de las intensidades de ejercicios a generar para cada nivel de dificultad. Como se tienen 5 objetivos de aprendizaje, hay 5 niveles de intensidad asociados a cada uno. El mejor indicador de desempeño del jugador es el tiempo que tomó en realizar un ejercicio. 

### Puntaje

Para empezar, se definirá un puntaje base que disminuirá a medida que el jugador tome más tiempo para contestar la pregunta. Nuestra función entonces estará dada de la siguiente forma:

\begin{equation*}
score = \Bigr(20 - \frac{tiempo}{3}\Bigl) * isCorrect
\end{equation*}

Siendo:
* __tiempo__: El tiempo que tardó el jugador en contestar la pregunta.
* __isCorrect__: Si la pregunta fue correcta (1) o incorrecta (0).

Para evitar un puntaje negativo, tal que no se penalice al jugador muy fuerte, lo que se busca es que si se equivoca o tarda demasiado en resolver el ejercicio no se le otorgue puntaje. 
El acercamiento que se propone es analizar por **grado** y **dificultad** los tiempos de los niños. 

In [7]:
#Graficar los tiempos



Como se puede ver en las gráficas, los tiempos por grado muestran una distribución normal pero sesgada a la derecha. No sería bueno utilizar la media en este escenario pues es muy sensible a los valores atípicos. Es por esto que se optará por utilizar la mediana, pues informa con más seguridad el centro de los datos.

In [8]:
#Sacar la mediana
#Mostrando mediana por grado y dificultad
timeMedianPerGrade = hartfortDF.groupby(['loID','grado'])['tiempo'].median();
timesMedianDF = timeMedianPerGrade.reset_index();
timesMedianDF.head(10)
#Realizar bulletplot

,loID,grado,tiempo
0,0,1,11.064110
1,0,2,7.257710
2,0,3,6.910470
3,0,4,5.397869
4,0,5,3.801245
5,1,1,7.570393
6,1,2,4.913258
7,1,3,3.818320
8,1,4,2.991299
9,1,5,2.406552


Con esto ahora sí, se procede a obtener los puntajes para cada ejercicio que responda el jugador.

In [9]:
def answerScore(answer):
    time = answer['tiempo'];
    timeLimit = int(timeMedianPerGrade[answer['loID'],answer['grado']]) + 2;
    if(time < timeLimit and answer['isCorrect'] == 1):
        return 20-(time/3);
    else:
        return 0;
    
hartfortDF['score'] = hartfortDF.apply(answerScore,axis=1);
hartfortDF.head(5)

,edad,genero,grado,loID,tiempo,userID,isCorrect,score
0,11,0,5,0,3.846108,-LaQZtfxo0QA8Ij62dk1,1,18.717964
1,10,0,5,0,2.995306,-LaQZurPwCF55Hiv18kH,1,19.001565
2,11,0,5,0,4.395411,-LaQZtfxo0QA8Ij62dk1,1,18.534863
3,10,0,5,0,2.422333,-LaQZurPwCF55Hiv18kH,1,19.192556
4,11,0,5,0,5.348062,-LaQZtfxo0QA8Ij62dk1,1,0.000000


### Intensidad de los LO

Ya teniendo el puntaje por ejercicio, el siguiente paso es realizar seguimiento a cada jugador para así saber cómo fue su desempeño en cada ejercicio.

\begin{eqnarray*}
General Score = \Bigl(\frac{\sum_{i=1}^{n}(Score_i)}{n}\Bigr)
\\
LO Intensity = \frac{General Score}{20}
\end{eqnarray*}

Donde
* __n__: El número de ejercicios que realizó un jugador en un objetivo específico.
* __Score__: Es el puntaje que obtuvo el jugador para ese ejercicio
* __Score General__: El promedio de puntos entre los ejercicios resueltos.
* __LOIntensity__: Es qué tan bien le fue al jugador para en un objetivo específico. Está medido en escala de 0 a 1, siendo 0 muy mal y 1 muy bien.

En síntesis, la intensidad se calcula como el promedio del puntaje de los ejercicios resueltos entre la cantidad máxima de puntos a obtener (20). Como se tienen 5 objetivos de aprendizaje, serán 5 intensidades a calcular.

In [10]:
def calculateIntensities(userPerformance):
    return (userPerformance.loc['score']/20);

#Grouping users and calculating the mean/average of the score 
usersTimesMean = hartfortDF.groupby(['userID','loID'])['score'].mean().reset_index();
usersTimesMean['intensity'] = usersTimesMean.apply(calculateIntensities,axis=1);
#Redondeo a 2 cifras
usersTimesMean.round({'intensity':1}).head(10)

,userID,loID,score,intensity
0,-LaQZtfxo0QA8Ij62dk1,0,11.329112,0.6
1,-LaQZtfxo0QA8Ij62dk1,1,15.341532,0.8
2,-LaQZtfxo0QA8Ij62dk1,2,15.325926,0.8
3,-LaQZtfxo0QA8Ij62dk1,3,18.892539,0.9
4,-LaQZtfxo0QA8Ij62dk1,4,3.845159,0.2
5,-LaQZurPwCF55Hiv18kH,0,11.480608,0.6
6,-LaQZurPwCF55Hiv18kH,1,15.371654,0.8
7,-LaQZurPwCF55Hiv18kH,2,19.137623,1.0
8,-LaQZurPwCF55Hiv18kH,3,18.635188,0.9
9,-LaQZurPwCF55Hiv18kH,4,7.230178,0.4


### Preparación del dataset

Luego de que se tiene las intensidades para cada usuario, lo que resta es organizar el dataframe para que sea más sencillo de pasar a la red neuronal. Lo que se hará es tomar la columna que tiene todas las intensidades y transformarlo a 5 columnas para cada usuario, donde estarán las intensidades asociadas a los respectivos niveles.

In [11]:
def getIntensityDF(intensityDF):
    LO0Array = []
    LO1Array = []
    LO2Array = []
    LO3Array = []
    LO4Array = []
    IDArray = []
    for i in range(0, 575, 5):
        IDArray.append(intensityDF.loc[i].userID)
        LO0Array.append(intensityDF.loc[i].intensity)
        LO1Array.append(intensityDF.loc[i+1].intensity)
        LO2Array.append(intensityDF.loc[i+2].intensity)
        LO3Array.append(intensityDF.loc[i+3].intensity)
        LO4Array.append(intensityDF.loc[i+4].intensity)
    d = {'userID':IDArray, 'LOI0': LO0Array,'LOI1': LO1Array,'LOI2': LO2Array,'LOI3': LO3Array,'LOI4': LO4Array}
    return pd.DataFrame(data=d)

#General user information
generalUserInfo = hartfortDF.groupby(['userID']).mean()
# SI Por algún motivo en jupyter se agrega una columna INDEX al groupby, hay que sacarla. Descomentar esta linea:
#generalUserInfo = generalUserInfo.drop(labels=['index','loID','tiempo','isCorrect','score'],axis=1);
#SI Index no aparece, usar esta:
generalUserInfo = generalUserInfo.drop(labels=['loID','tiempo','isCorrect','score'],axis=1);

#Adding the LOs intensity to the dataset
#This intensity dice qué tan bien va para ese LO. Si se interpreta que un niño tiene
#0.6 en el LO1, significa que necesita una intensidad de 1-0.6 -> 0.4 de ejercicios
#para la siguiente pasada.
LOIntensitiesPerUser = getIntensityDF(usersTimesMean);
LOIntensitiesPerUser = LOIntensitiesPerUser.set_index('userID');
LOIntensitiesPerUser = LOIntensitiesPerUser.round(1);
generalUserInfo[['LOIN0','LOIN1','LOIN2','LOIN3','LOIN4']] = LOIntensitiesPerUser[['LOI0','LOI1','LOI2','LOI3','LOI4']];
generalUserInfo.head(5)

,edad,genero,grado,LOIN0,LOIN1,LOIN2,LOIN3,LOIN4
userID,,,,,,,,
-LaQZtfxo0QA8Ij62dk1,11,0,5,0.6,0.8,0.8,0.9,0.2
-LaQZurPwCF55Hiv18kH,10,0,5,0.6,0.8,1.0,0.9,0.4
-LaQZvlOUvQcKzVILey9,10,0,5,0.8,1.0,0.6,0.8,0.2
-LaQZxvCZUvf9J0938NI,10,0,5,0.9,1.0,0.8,0.9,0.5
-LaQ_08Mr96o00YYSj4_,10,1,5,0.6,0.8,0.6,0.6,0.2


## Construcción de la Red Neuronal

Luego de que se tiene el dataset listo, se procede a partir entre variables independientes e independientes. Este proceso es para pasar los datos a la red neuronal.


### Arquitectura

El primer modelo de red neuronal que se diseñará se encargará de nivelar a un nuevo jugador. Cuando un nuevo jugador ingrese, la red deberá estimar las intensidades de los niveles de aprendizaje con base en su edad, el grado que cursa y género.
<img height=600, width=600, src="NewPlayerNeuralNetwork.png">

En el futuro se considerará la escuela también se considerará.

In [12]:
#Splitting the dataset
#Let X be Edad,Grado and Genero
X = generalUserInfo.drop(labels=['LOIN0','LOIN1','LOIN2','LOIN3','LOIN4'],axis=1);
#Let Y be the 5 intensities
Y = generalUserInfo[['LOIN0','LOIN1','LOIN2','LOIN3','LOIN4']].copy()

Se utilizará el 80% de los datos para entrenamiento y 20% para probar. Además, se deben escalar los datos para evitar la dominancia entre variables.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
sc = StandardScaler() #Must be scaled to avoid variable domination
#Transforming dataset
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Steven\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """


### Construcción de la red

Utilizando la librería de Keras, se definirá la red neuronal como una secuencia de capas. Siguiendo el gráfico explicado en la sección anterior, tendrémos 3 capas: 

#### Input Layer
Recibirá los datos de entrada: Edad, Género y Grado. Por tanto, la dimensión de esta red debe ser 3

#### Hidden Layer
Recibirá los datos de entrada y realizará diferentes combinaciones entre ellos buscandor relaciones para obtener las intensidades. Se utilizará la funcion de activcación de _Rectifier Linear Unit(ReLU)_ porque tradicionalmente es la que mejor performance da y hace que la red sea más fácil y rápido de entrenar a la red neuronal. Entre otras ventajas que presenta sobre otras funciones (sigmoide y tanh) es que esta no es tan sensible a los cambios. La cantidad de nodos que se llevará esta red son 10. Esto es porque, como no hay una respuesta precisa de cuantas unidades se deben usar, nuestra aproximación fue probar diferentes cantidades (5 10 15 20) y tomar la que mejor resultados brindase.

#### Output Layer
Contará con los 5 outputs de niveles de intensidad.

#### Kernel Initializers
Son las distribuciones con las que se asignarán los pesos a las aristas de la red neuronal. Entre distribución uniforme y normal no hay mucha diferencia en cuanto a qué tanto influyen en el modelo, pues ambos dan buenos resultados. Tomamos entonces una inicialización uniforme para las aristas de la hidden layer y una normal para la output layer.

#### Optimizer
Es la función para encontrar los valores óptimos. Como estamos haciendo una regresión, utilizamos el Stochastic Gradient Descent (SGD). 

#### Loss
La función de optimización de los pesos usaremos el error cuadrático medio (MSE). Esto principalmente por ser una regresión y se buscan valores que sean lo más aproximados posible.

#### Metrics
Para medir la precisión del modelo serán el MSE y el error absoluto medio (MAE).

### Entrenamiento de la red

#### batch_size
La cantidad de muestras que tomará la red para entrenarse al momento de hacer una propagación (paso entre input - hidden - output) para luego re calcular los pesos y ajustarse. Lo ideal es que no sea muy pequeño para que el movimiento del gradiente pueda dar resultados más precisos, pero tampoco muy grande para que no consuma mucha memoria.

#### epochs
La cantidad de veces que repetirá todo el proceso de entrenamiento con el dataset. 1 época se define como una pasada completa de todo el dataset, tras haber hecho proceso de forward y backward propagation.

Entonces para nuestra red que consta de 92 ejemplares para entrenar, analizará de 5 en 5 para calcular los pesos. Cuando llegue a los 92, habrá finalizado una época. Entonces vuelve a repetirse 49 épocas más para mejorar su precisión.


In [14]:
#Building the ANN
classifier = Sequential();
classifier.add(Dense(input_dim=3, activation='relu', kernel_initializer='uniform', units=10));
classifier.add(Dense(kernel_initializer='normal',units=5))
classifier.compile(optimizer='sgd', loss='mean_squared_error', metrics=['mse','mae']);
classifier.fit(X_train, y_train, batch_size=5, epochs=50)
y_pred = classifier.predict(X_test);

Epoch 1/50
92/92 [==============================] - 19s 204ms/step - loss: 0.2664 - mean_squared_error: 0.2664 - mean_absolute_error: 0.4224
Epoch 2/50
92/92 [==============================] - 0s 602us/step - loss: 0.2392 - mean_squared_error: 0.2392 - mean_absolute_error: 0.4000
Epoch 3/50
92/92 [==============================] - 0s 596us/step - loss: 0.2160 - mean_squared_error: 0.2160 - mean_absolute_error: 0.3792
Epoch 4/50
92/92 [==============================] - 0s 618us/step - loss: 0.1957 - mean_squared_error: 0.1957 - mean_absolute_error: 0.3598
Epoch 5/50
92/92 [==============================] - 0s 791us/step - loss: 0.1787 - mean_squared_error: 0.1787 - mean_absolute_error: 0.3422
Epoch 6/50
92/92 [==============================] - 0s 640us/step - loss: 0.1637 - mean_squared_error: 0.1637 - mean_absolute_error: 0.3257
Epoch 7/50
92/92 [==============================] - 0s 737us/step - loss: 0.1510 - mean_squared_error: 0.1510 - mean_absolute_error: 0.3128
Epoch 8/50
92/92 [=

Finalmente, esta configuración con la red neuronal que otorga un error de 0.21 se guarda en un archivo. Este contendrá los pesos y toda la arquitectura que definimos para el funcionamiento de sus capas, tal que cuando se vuelva a utilizar solo sea cargar la información y no haya que volver a entrenar el modelo.

In [15]:
filename = 'neuralnet'
outfile = open(filename,'wb')
pickle.dump(classifier,outfile)
outfile.close()